<a href="https://colab.research.google.com/github/maquico/IA-IDS330/blob/main/ids330_wumpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mundo del Wumpus

**Reglas de Juego**

1. El agente solo se puedo mover arriba, abajo, izquiera y derecha. No se puede mover en diagonal.
2. Si hay un agujero en una casilla adyacente (excluyendo las diagonales), **el agente siente brisa**.
3. Si el Wumpus está en una casilla adyacente (excluyendo las diagonales), **el agente siente un mal olor**.
4. El agente puede eliminar el Wumpus si sabe donde está.
5. Si el oro se encuentra en la casilla donde está el agente, **el agente siente un brillo**, y puedo el coger el oro.
6. El agente sabe cuando hay una pared. El agente no volarse la pared.
7. Se gana cogiendo el oro y volviendo a la casilla original.

**Más específico**
- El mundo será 5x5.
- El agente siempre va a comenzar en la casilla (0, 0): la esquina superior izquierda.
- Solamente habrá un Wumpus.
- Habrá de 2 a 5 agujeros.
- Las casillas adyacentes al (0, 0) siempre estarán libres de agujeros o wumpus.
- Solamente un oro.

## Sistema Experto

### Definir instancia del mundo

In [ ]:
HEIGHT = 5
WIDTH = 5
NUM_PITS = 2
NUM_WUMPUS = 1
NUM_GOLD = 1
INITIAL_COORD_AGENT = (0, 0)

In [ ]:
import random
import itertools

random.seed(2024)


def get_adjacent_caves(coord: tuple, height: int, width: int) -> list:

    w, h = coord[0], coord[1]
    adjacent_caves = []

    if h - 1 >= 0:
        adjacent_caves.append((w, h-1))

    if h + 1 < height:
        adjacent_caves.append((w, h+1))

    if w - 1 >= 0:
        adjacent_caves.append((w-1, h))

    if w + 1 < width:
        adjacent_caves.append((w+1, h))

    return adjacent_caves


class WumpusWorld():
    def __init__(self, initial_coord_agent: tuple, height: int, width: int, num_pits: int, num_wumpus: int, num_gold: int):
        self.height = height
        self.width = width
        self.num_pits = num_pits
        self.num_wumpus = num_wumpus
        self.num_gold = num_gold

        '''
        tamaño: 5x5 ==> [(0,0), (0,1), ..., (4,4)], 25 coords
        '''
        self.coords = itertools.product(list(range(height)), list(range(width)))
        self.coords = list(self.coords)

        # Seleccionando coordenadas iniciales

        coords_except_entrance = [c for c in self.coords if c not in [(0,0), (0,1), (1,0)]]
        select_coords = random.sample(coords_except_entrance, num_pits + num_wumpus + num_gold)
        # [(2,2), (1,3), (3,4), (3,1)]
        self.coord_agent = initial_coord_agent
        self.coord_wumpus = select_coords[0] # e.g., (2,2)
        self.coord_gold = select_coords[1] # e.g., (1,3)
        self.coord_pits = select_coords[2:] # e.g., [(3,4), (3,1)]
        self.is_wumpus_alive = True

        # Construyendo el mundo
        self.world = []
        for h in range(self.height):
            self.world.append(['*'] * self.width)

        self.update_world(self.coord_agent)
        self.breeze_coords = self.get_breeze_coords()



    def update_world(self, new_coord_agent):

        self.world[self.coord_agent[0]][self.coord_agent[1]] = '*'  # cueva

        self.coord_agent = new_coord_agent

        if self.is_wumpus_alive:
            self.world[self.coord_wumpus[0]][self.coord_wumpus[1]] = 'w'  # wumpus
        else:
            self.world[self.coord_wumpus[0]][self.coord_wumpus[1]] = '*'

        self.world[self.coord_gold[0]][self.coord_gold[1]] = 'g'  # oro

        for c in self.coord_pits:
            self.world[c[0]][c[1]] = 'p'  # agujero

        self.world[self.coord_agent[0]][self.coord_agent[1]] = '☻'

    def get_breeze_coords(self):

        breeze_coords = []

        for c in self.coord_pits:
            adjacent_caves = get_adjacent_caves(c, self.height, self.width)
            adjacent_caves = [a for a in adjacent_caves if a not in self.coord_pits]
            breeze_coords.extend(adjacent_caves)

        return list(set(breeze_coords))

    def get_smell_coords(self):

        if self.is_wumpus_alive:
            return get_adjacent_caves(self.coord_wumpus, self.height, self.width)
        else:
            return None

    def get_gold_coord(self):
        return self.coord_gold

    def print_world(self):
        for row in self.world:
            print(''.join(row))



wumpus_world = WumpusWorld(INITIAL_COORD_AGENT, HEIGHT, WIDTH, NUM_PITS, NUM_WUMPUS, NUM_GOLD)
wumpus_world.print_world()

☻****
***g*
**p**
***w*
*p***


In [ ]:
# AGENTE

def print_believe(believe: dict, height: int, width: int):
    world = []
    for h in range(height):
        world.append(['-'] * width)

    for key, item in believe.items():
        world[key[0]][key[1]] = item

    for row in world:
        print(''.join(row))


class Agent():
    def __init__(self, initial_position=(0,0)):
        self.position = initial_position
        self.is_alive = True
        self.has_gold = False
        self.current_senses = []
        self.previous_cave = None
        self.memory = [initial_position]
        self.believe = {}


    def sense_cave(self, breeze_coords, smell_coords, gold_coord):

        self.current_senses = []

        if self.position == gold_coord:
            self.current_senses.append('g')
            self.has_gold = True  # Toma el oro

        if self.position in breeze_coords:
            self.current_senses.append('breeze')

        if self.position in smell_coords:
            self.current_senses.append('smell')


    def update_believe(self, adjacent_caves):

        # BASE DE CONOCIMIENTO

        self.believe[self.position] = '*'

        for cave in adjacent_caves:

            # Si es la casilla anterior, obviar
            if cave == self.previous_cave:
                continue

            if cave not in self.believe.keys():
                self.believe[cave] = ''

            if len(self.current_senses) == 0:
                # Si no siente nada, pues las casillas adyacentes son seguras
                self.believe[cave] = '*'
            else:
                if 'breeze' in self.current_senses:
                    if self.believe[cave] == '':
                        self.believe[cave] = 'p'
                    elif '*' not in self.believe[cave]:
                        if 'p' in self.believe[cave]:
                            self.believe[cave] = 'p'
                else:
                    if 'p' in self.believe[cave]:
                        if 'w' in self.believe[cave]:
                            self.believe[cave] = self.believe[cave].replace('p', '')
                        else:
                            self.believe[cave] = '*'

                if 'smell' in self.current_senses:
                    if self.believe[cave] == '':
                        self.believe[cave] += 'w'
                    elif '*' not in self.believe[cave]:
                        if 'w' in self.believe[cave]:
                            self.believe[cave] += 'w'
                else:
                    if 'w' in self.believe[cave]:
                        if 'p' in self.believe[cave]:
                            self.believe[cave] = self.believe[cave].replace('w', '')
                        else:
                            self.believe[cave] = '*'


    def step(self, adjacent_caves):

        if self.has_gold:
            # self.go_back_home()
            return 'win'

        possible_caves = []
        for cave in adjacent_caves:
            if (self.believe[cave] == '*') and (cave != self.previous_cave):
                possible_caves.append(cave)

        if len(possible_caves) == 0:
            if self.previous_cave == None:
                print('Cannot move!')
                return 'fail'
            else:
                self.position, self.previous_cave = self.previous_cave, self.position

        else:
            self.previous_cave = self.position
            self.position = random.choice(possible_caves)

        self.memory.append(self.previous_cave)

        return 'step'


agent = Agent()


In [ ]:
import time

random.seed(2024)
wumpus_world = WumpusWorld(INITIAL_COORD_AGENT, HEIGHT, WIDTH, NUM_PITS, NUM_WUMPUS, NUM_GOLD)
agent = Agent()

breeze_coords = wumpus_world.get_breeze_coords()
smell_coords = wumpus_world.get_smell_coords()
gold_coord = wumpus_world.get_gold_coord()

wumpus_world.print_world()

for j in range(20):
    print(f"\nIter: {j}")
    adjacent_caves = get_adjacent_caves(agent.position, height=HEIGHT, width=WIDTH)
    agent.sense_cave(breeze_coords, smell_coords, gold_coord)
    agent.update_believe(adjacent_caves)
    print_believe(agent.believe, HEIGHT, WIDTH)
    status = agent.step(adjacent_caves)
    print('=====')
    wumpus_world.update_world(agent.position)
    wumpus_world.print_world()

    if status == 'win':
        print('Win!')
        break



    # time.sleep(1)


☻****
***g*
**p**
***w*
*p***

Iter: 0
**---
*----
-----
-----
-----
=====
*☻***
***g*
**p**
***w*
*p***

Iter: 1
***--
**---
-----
-----
-----
=====
*****
*☻*g*
**p**
***w*
*p***

Iter: 2
***--
***--
-*---
-----
-----
=====
*****
***g*
*☻p**
***w*
*p***

Iter: 3
***--
***--
p*p--
-p---
-----
=====
*****
*☻*g*
**p**
***w*
*p***

Iter: 4
***--
***--
p*p--
-p---
-----
=====
*****
**☻g*
**p**
***w*
*p***

Iter: 5
***--
***p-
p*p--
-p---
-----
=====
**☻**
***g*
**p**
***w*
*p***

Iter: 6
****-
***p-
p*p--
-p---
-----
=====
***☻*
***g*
**p**
***w*
*p***

Iter: 7
*****
****-
p*p--
-p---
-----
=====
*****
***☻*
**p**
***w*
*p***

Iter: 8
*****
****
p*p-
-p---
-----
=====
*****
***☻*
**p**
***w*
*p***
Win!
